In [ ]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
import matplotlib as mpl

def process_sentinel2_image(input_path, output_path):
    """
    处理 Sentinel-2 图像:
    1. 读取5波段TIFF图像
    2. 将数据范围从0-10000压缩到0-255
    3. 转换为RGB三通道图像
    4. 保存处理结果
    """
    # 1. 设置中文字体支持
    plt.rcParams['font.sans-serif'] = ['SimHei', 'Microsoft YaHei', 'KaiTi', 'Arial Unicode MS']
    plt.rcParams['axes.unicode_minus'] = False
    
    # 2. 读取原始TIFF图像
    with rasterio.open(input_path) as src:
        # 读取所有5个波段
        bands = src.read()
        profile = src.profile
        
        # 3. 数据压缩 (0-10000 → 0-255)
        # 添加裁剪确保在0-255范围内
        compressed = np.clip(bands / 10000.0 * 255, 0, 255).astype(np.uint8)
        
        # 4. 选择RGB波段 (通常为B2, B3, B4)
        rgb_indices = [2, 1, 0]  # 红、绿、蓝波段索引
        rgb_image = compressed[rgb_indices, :, :]
        
        # 5. 更新元数据为RGB三通道
        profile.update(
            count=3,
            dtype=np.uint8
        )
        
        # 6. 保存处理后的图像
        with rasterio.open(output_path, 'w', **profile) as dst:
            dst.write(rgb_image)
    
    print(f"处理完成! 结果已保存至: {output_path}")
    
    # 7. 显示处理前后的对比 (修复显示问题)
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 10))
    
    # 原始图像显示
    with rasterio.open(input_path) as src:
        # 创建真彩色合成
        rgb_bands = [src.read(i) for i in [3, 2, 1]]  # 红、绿、蓝波段
        
        # 缩放并裁剪到0-255
        rgb_display = np.stack([
            np.clip(band / 10000.0 * 255, 0, 255).astype(np.uint8)
            for band in rgb_bands
        ])
        
        # 转置为(height, width, bands)
        rgb_display = np.moveaxis(rgb_display, 0, -1)
        
        # 显示图像
        ax1.imshow(rgb_display)
        ax1.set_title('原始图像')
    
    # 处理后的图像显示
    with rasterio.open(output_path) as dst:
        # 读取并转置为(height, width, bands)
        processed_rgb = dst.read()
        processed_rgb = np.moveaxis(processed_rgb, 0, -1)
        
        ax2.imshow(processed_rgb)
        ax2.set_title('处理后RGB')
    
    # 8. 保存和显示
    plt.tight_layout()
    plt.savefig('comparison.png', dpi=300, bbox_inches='tight')
    plt.show()

# 使用示例
if __name__ == "__main__":
    input_tif = "2020_0427_fire_B2348_B12_10m_roi.tif"  # 输入文件路径
    output_tif = "processed_rgb.tif"   # 输出文件路径
    
    process_sentinel2_image(input_tif, output_tif)

处理完成! 结果已保存至: processed_rgb.tif
